# Lab 4: Spark Dataframes and SQL (Part I)

In this lab we are practicing data transformation and processing using Spark Dataframe and SQL APIs.
In the first part, we practice using the London crime data set. We start by loading the data from the file system and how withe a few options we tell Spark the nature of the data and its storage format and how Spark tries to automatically infer a schema for the data, something that would have required explicit transformations using low-level Spark RDDS.

We practice data cleaning by dropping rows with empty cells, unwanted columns. Next, we show a limited number of records, select unique values of a specific column, do aggregation overall the data or by a specific attribute(s), etc. We utilize the richset of DF APIs to try aggregation, sampling, filtering, projection and so on.

In the second part, we will play with another data set about soccer players. Here, will use SQL and practice the joining of data sets.

### Analyzing London crime statistics

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

#### Creating a Spark session

* Encapsulates SparkContext and the SQLContext within it

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Analyzing London crime data") \
    .master('spark://spark-master:7077') \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .getOrCreate()

22/09/19 21:51:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/09/19 21:51:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/19 21:51:24 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [5]:
spark

#### Reading external data as a dataframe
We need to load the d|ata from disk. The data is stored as CSV file. With simple instructions, we tell Spark that the data is in CSV and has a header row. The header row is used by Spark to infer the data schema.

If necessary, unzip london_crime_by_lsoa.zip 

In [6]:
data = spark.read\
            .format("csv")\
            .option("header", "true")\
            .load("data/london_crime_by_lsoa.csv")

22/09/19 21:53:25 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


In [7]:
data.printSchema()

root
 |-- lsoa_code: string (nullable = true)
 |-- borough: string (nullable = true)
 |-- major_category: string (nullable = true)
 |-- minor_category: string (nullable = true)
 |-- value: string (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)



What is the length(count) of data?

In [8]:
#Code here
# Remove before sharing with students
data.count()

13490604

Show just five records from the data set

In [9]:
#Remove before sharing with students
data

lsoa_code,borough,major_category,minor_category,value,year,month
E01001116,Croydon,Burglary,Burglary in Other...,0,2016,11
E01001646,Greenwich,Violence Against ...,Other violence,0,2016,11
E01000677,Bromley,Violence Against ...,Other violence,0,2015,5
E01003774,Redbridge,Burglary,Burglary in Other...,0,2016,3
E01004563,Wandsworth,Robbery,Personal Property,0,2008,6
E01001320,Ealing,Theft and Handling,Other Theft,0,2012,5
E01001342,Ealing,Violence Against ...,Offensive Weapon,0,2010,7
E01002633,Hounslow,Robbery,Personal Property,0,2013,4
E01003496,Newham,Criminal Damage,Criminal Damage T...,0,2013,9
E01004177,Sutton,Theft and Handling,Theft/Taking of P...,1,2016,8


#### Cleaning data
* Drop rows which have null (empty) values
* Drop columns which we do not use in our analysis(lsoa_code column)

In [6]:
#code here
#Remove before sharing with students
data.dropna()

DataFrame[lsoa_code: string, borough: string, major_category: string, minor_category: string, value: string, year: string, month: string]

In [10]:
# code below to drop column lsoa_code
data = data.drop('lsoa_code')

data

borough,major_category,minor_category,value,year,month
Croydon,Burglary,Burglary in Other...,0,2016,11
Greenwich,Violence Against ...,Other violence,0,2016,11
Bromley,Violence Against ...,Other violence,0,2015,5
Redbridge,Burglary,Burglary in Other...,0,2016,3
Wandsworth,Robbery,Personal Property,0,2008,6
Ealing,Theft and Handling,Other Theft,0,2012,5
Ealing,Violence Against ...,Offensive Weapon,0,2010,7
Hounslow,Robbery,Personal Property,0,2013,4
Newham,Criminal Damage,Criminal Damage T...,0,2013,9
Sutton,Theft and Handling,Theft/Taking of P...,1,2016,8


#### Boroughs included in the report

<b>select the unique boroughs we have in the 'borough' column.</b>

In [11]:
# code here
# Remove before sharing with students
unique_boroughs = data.select('borough').distinct()
        
unique_boroughs

borough
Croydon
Wandsworth
Bexley
Lambeth
Barking and Dagenham
Camden
Greenwich
Newham
Tower Hamlets
Barnet


<b>Show how many unique boroughs are there in the data.</b>

In [12]:
# code here
# Remove before sharing with students
unique_boroughs.count()

33

<b>Get the data related to borough "Hackney" only.</b>

In [13]:
# code here
# Remove before sharing with students
hackney_data = data.filter(data.borough == "Hackney")

hackney_data

borough,major_category,minor_category,value,year,month
Hackney,Criminal Damage,Criminal Damage T...,0,2011,6
Hackney,Violence Against ...,Harassment,1,2013,2
Hackney,Criminal Damage,Other Criminal Da...,0,2011,7
Hackney,Violence Against ...,Wounding/GBH,0,2013,12
Hackney,Theft and Handling,Other Theft Person,0,2016,8
Hackney,Burglary,Burglary in a Dwe...,2,2008,5
Hackney,Robbery,Business Property,0,2016,7
Hackney,Theft and Handling,Theft/Taking of P...,0,2009,12
Hackney,Drugs,Drug Trafficking,0,2014,4
Hackney,Theft and Handling,Handling Stolen G...,0,2014,6


<b>Get data related to years 2015 and 2016.</b>

<sub>Hint: use isin()</sub>

<sub>Hint: use *sample* function to select a fraction of 10% of the data</sub>

In [14]:
# Code here
# Remove before sharing with students
data_2015_2016 = data.filter(data.year.isin('2016', '2015'))

data_2015_2016.sample(fraction=0.1)

borough,major_category,minor_category,value,year,month
Hounslow,Criminal Damage,Criminal Damage T...,0,2015,2
Hackney,Theft and Handling,Other Theft Person,0,2016,8
Newham,Theft and Handling,Theft/Taking of P...,0,2016,3
Kensington and Ch...,Other Notifiable ...,Other Notifiable,0,2015,5
Hillingdon,Violence Against ...,Other violence,0,2016,11
Richmond upon Thames,Violence Against ...,Common Assault,0,2015,3
Greenwich,Robbery,Business Property,0,2015,12
Havering,Violence Against ...,Harassment,1,2016,8
Hammersmith and F...,Burglary,Burglary in Other...,0,2015,3
Barnet,Theft and Handling,Handling Stolen G...,0,2016,4


<b>Get all the data related to year 2014 and forward.</b>

In [16]:
#code here
# Remove before sharing with students
data_2014_onwards = data.filter (data.year >= 2014)

data_2014_onwards.sample(fraction=0.1)

borough,major_category,minor_category,value,year,month
Richmond upon Thames,Robbery,Personal Property,0,2014,1
Southwark,Theft and Handling,Theft From Shops,4,2016,8
Barnet,Violence Against ...,Other violence,0,2015,12
Newham,Burglary,Burglary in Other...,0,2015,2
Croydon,Robbery,Personal Property,0,2016,1
Kensington and Ch...,Robbery,Personal Property,1,2015,7
Tower Hamlets,Robbery,Personal Property,0,2016,10
Ealing,Theft and Handling,Theft/Taking Of M...,0,2016,1
Croydon,Robbery,Business Property,0,2014,6
Wandsworth,Theft and Handling,Handling Stolen G...,0,2016,2


<b>Get the count of crimes for each borough.</b>

In [17]:
# Code here
# Remove before sharing with students
borough_crime_count = data.groupBy('borough').count()
    
borough_crime_count

borough,count
Croydon,602100
Wandsworth,498636
Bexley,385668
Lambeth,519048
Barking and Dagenham,311040
Camden,378432
Greenwich,421200
Newham,471420
Tower Hamlets,412128
Barnet,572832


<b> Get the total convictions per borough.</b>

In [18]:
# Code here
# Remove before sharing with students
borough_conviction_sum = data.groupBy('borough')\
                             .agg({"value":"sum"})

borough_conviction_sum

borough,sum(value)
Croydon,260294.0
Wandsworth,204741.0
Bexley,114136.0
Lambeth,292178.0
Barking and Dagenham,149447.0
Camden,275147.0
Greenwich,181568.0
Newham,262024.0
Tower Hamlets,228613.0
Hounslow,186772.0


<b>Rename the column of sum(value) to be "convictions".</b>

<sub> Hint: use built-in function *withColumnRenamed*.</sub>

In [19]:
# Code here
# Remove before sharing with students
borough_conviction_sum = borough_conviction_sum.withColumnRenamed('sum(value)', 'convictions')

borough_conviction_sum

borough,convictions
Croydon,260294.0
Wandsworth,204741.0
Bexley,114136.0
Lambeth,292178.0
Barking and Dagenham,149447.0
Camden,275147.0
Greenwich,181568.0
Newham,262024.0
Tower Hamlets,228613.0
Hounslow,186772.0


#### Per-borough convictions expressed in percentage

<b> Get the sum of overall convictions.</b>

In [20]:
# Code here
# Remove before sharing with students
total_borough_convictions = borough_conviction_sum.agg({"convictions":"sum"})

total_borough_convictions

sum(convictions)
6447758.0


<b> Extract the total convictions into a variable.</b>

In [21]:
total_convictions = total_borough_convictions.collect()[0][0]

<b> Add a new column which contains the percentage of convictions for each borough.</b>

<sub> Hint: will use the *withColumn* built-in function to add a new column to the DF.</sub>

<sub> Hint: We can use a rounding function to make the percentage rounded to two decimal places. For this, use the pyspark.sql.functions package.</sub>

In [22]:
import pyspark.sql.functions as func

In [23]:
# Code here
# Remove before sharing with students
borough_percentage_contribution = borough_conviction_sum.withColumn('% contribution',func.round((borough_conviction_sum.convictions/total_convictions)*100,2))

borough_percentage_contribution.printSchema()

root
 |-- borough: string (nullable = true)
 |-- convictions: double (nullable = true)
 |-- % contribution: double (nullable = true)



<b> Order the data frame borough_percentage_contribution by the third added column in descending order  and show your results.</b>

In [28]:
# Code here
# Remove before sharing with students
df = borough_percentage_contribution.orderBy('% contribution', ascending=False)
df

borough,convictions,% contribution
Westminster,455028.0,7.06
Lambeth,292178.0,4.53
Southwark,278809.0,4.32
Camden,275147.0,4.27
Newham,262024.0,4.06
Croydon,260294.0,4.04
Ealing,251562.0,3.9
Islington,230286.0,3.57
Tower Hamlets,228613.0,3.55
Brent,227551.0,3.53


<b>Project the "year" column.</b>

In [29]:
# Code here
# Remove before sharing with students
year_df = data.select('year')
year_df

year
2016
2016
2015
2016
2008
2012
2010
2013
2013
2016


<b>Show the minimum year.</b>

In [30]:
# Code here
# Remove before sharing with students
year_df.agg({"year":"min"})

min(year)
2008


<b> Show the maximum year.</b>

In [ ]:
# Code here
# Remove before sharing with students
year_df.agg({"year":"max"})

In [32]:
#you can get summary using describe
year_df.describe()

summary,year
count,13490604
mean,2012.0
stddev,2.581988993167432
min,2008
max,2016


#### Distribution of crime across boroughs in a particular year

Below is a helper function that shall be used to describe each year. This function is expecting a value for the year and it wil run against the original data DF.

In [33]:
def describe_year(year):
    yearly_details = data.filter(data.year == year)\
                         .groupBy('borough')\
                         .agg({'value':'sum'})\
                         .withColumnRenamed("sum(value)","convictions")
    
    borough_list = [x[0] for x in yearly_details.toLocalIterator()]
    convictions_list = [x[1] for x in yearly_details.toLocalIterator()]
  
    plt.figure(figsize=(33, 10)) 
    plt.bar(borough_list, convictions_list)
    
    plt.title('Crime for the year: ' + year, fontsize=30)
    plt.xlabel('Boroughs',fontsize=30)
    plt.ylabel('Convictions', fontsize=30)

    plt.xticks(rotation=90, fontsize=30)
    plt.yticks(fontsize=30)
    plt.autoscale()
    plt.show()

In [ ]:
describe_year('2014')